# Data Cleaning and Preparation

In [1]:
import pandas as pd
import numpy as np

## Data Transformation

### Generali

#### Di Modifica 

- **Rinominare gli assi**

In [108]:
data = pd.DataFrame(
    np.arange(12).reshape((3,4)),
    index = ['Ohio', 'Colorado', 'New York'],
    columns = ['one', 'two', 'three', 'four']
)

In [109]:
def transform(x):
    return x[:4].upper()

In [110]:
# mapping the index as a Series
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [111]:
# renaming the axis (in place)
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [113]:
# renaming the axis (transformed)
data.rename(index = str.title, columns = str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [ ]:
# renaming a subset of the axis
data.rename(index = {'OHIO' : 'INDIANA'}, # Ohio -> Indiana
        columns = {'three' : 'peakaboo'}) # three -> peakaboo

,one,two,peakaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


- **Rimpiazzare i Valori**

In [116]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [ ]:
# rimpiazzare un valore 
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [ ]:
# rimpiazzare piu valori
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [120]:
# # rimpiazzare i valori con un valore specifico 
data.replace([-999, -1000], [np.nan, 0])
data.replace({-999 : np.nan, -1000 : 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

- **Trasformare i dati con una funzione o una mappa**

> **Utilita**
> 
> Serve per applicare delle trasformazioni elemento per elemento (element-wise) ai dati nel DataFrame

In [126]:
data = pd.DataFrame(
    {
        'food': ['bacon', 'pulled pork', 'bacon', 'pastrami', 'corned beef', 'bacon', 'pastrami', 'honey ham', 'nova lox'],
        'ounces' : [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
    }
)
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [ ]:
# definizione della mappa 
meat_to_animal = {
    'bacon' : 'pig',
    'pulled pork' : 'pig',
    'pastrami' : 'cow', 
    'corned beef' : 'cow',
    'honey ham' : 'pig',
    'nova lox' : 'salmon'
}

In [ ]:
# aggiunta della colonna risultato al dataframe originale 
data['animal'] = data['food'].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,noa lox,6.0,NaN


In [128]:
# stesso processo con una funzione 
def get_animal(x):
    return meat_to_animal[x]

data['food'].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

#### Di Rimozione

- **Rimozione dei Duplicati**

In [88]:
data = pd.DataFrame({
    'k1' : ['one', 'two']*3 + ['two'],
    'k2' : [1,1,2,3,3,4,4]
})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [ ]:
# filtrare le righe per valori duplicati 
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [90]:
# rimuovere i dupicati 
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [92]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [93]:
# rimuovere i duplicati per colonna 
data.drop_duplicates(subset = ['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


- **Individuzione e rimozione degli Outlier**

In [96]:
np.random.seed(42)
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.030624,0.024828,-0.008255,0.030086
std,0.963919,1.011884,1.006075,1.006964
min,-3.019512,-2.896255,-3.241267,-2.991136
25%,-0.612942,-0.677037,-0.675299,-0.670871
50%,0.056187,0.020210,-0.007509,0.021158
75%,0.664881,0.693881,0.642282,0.695878
max,3.243093,3.852731,3.152057,3.926238


In [97]:
col = data[2]
col[col.abs() > 3]

65    -3.241267
119    3.078881
995    3.152057
Name: 2, dtype: float64

In [ ]:
# individuare gli outlier
data[(data.abs() > 3).any(axis = 'columns')]

,0,1,2,3
52,0.515048,3.852731,0.570891,1.135566
65,-0.926930,-0.059525,-3.241267,-1.024388
119,0.576557,0.311250,3.078881,1.119575
403,0.883110,-0.077837,-0.180480,3.193108
489,-2.135674,3.137749,1.056057,0.223239
506,-3.019512,0.183850,1.800511,1.238946
576,1.995667,3.109919,0.606723,-0.183197
723,0.768207,0.215397,0.508269,3.926238
929,3.243093,2.307916,-0.181449,-0.106337
995,1.362563,1.640615,3.152057,-1.123494


In [ ]:
# clipping: ad ogni outlier assegna un valore (3, -3) a seconda del suo segno
data[data.abs() > 3] = np.sign(data)*3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.030400,0.023728,-0.008245,0.028967
std,0.963077,1.008264,1.004621,1.003207
min,-3.000000,-2.896255,-3.000000,-2.991136
25%,-0.612942,-0.677037,-0.675299,-0.670871
50%,0.056187,0.020210,-0.007509,0.021158
75%,0.664881,0.693881,0.642282,0.695878
max,3.000000,3.000000,3.000000,3.000000


### Specifiche

- Binning 

In [129]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]

In [ ]:
# output: special Categorical object
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [132]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [133]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [134]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [135]:
pd.value_counts(age_categories)

/tmp/ipykernel_7877/3010498523.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(age_categories)


(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

In [138]:
# nomi specifici per gli intervalli 
names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels = names, precision = 2) # classification limited to the second digit after the comma

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

Un metodo piu statistico per effettuare il *binning* dei dati e quello implementato con `qcut`. Questo metodo genere i bin in funzione dei quantili campionari della popolazione. Questo ha il vantaggio di assicurare che in ogni bin si trovano piu o meno lo stesso numero di dati. 

In [140]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision = 2)
quartiles

[(-3.19, -0.68], (-0.68, -0.018], (-0.018, 0.64], (-0.018, 0.64], (-3.19, -0.68], ..., (-0.68, -0.018], (0.64, 3.11], (0.64, 3.11], (0.64, 3.11], (-3.19, -0.68]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.19, -0.68] < (-0.68, -0.018] < (-0.018, 0.64] < (0.64, 3.11]]

In [141]:
pd.value_counts(quartiles)

/tmp/ipykernel_7877/3472704981.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(quartiles)


(-3.19, -0.68]     250
(-0.68, -0.018]    250
(-0.018, 0.64]     250
(0.64, 3.11]       250
Name: count, dtype: int64

In [ ]:
# utilizzare quantili specifici 
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1]).value_counts()

(-3.178, -1.353]     100
(-1.353, -0.0182]    400
(-0.0182, 1.174]     400
(1.174, 3.113]       100
Name: count, dtype: int64

> **Attenzione**
>
> I quantili sono considerati numeri fra 0 ed 1. Si possono pensare questi valori come frazioni della totalita dei dati. 


- Permutazioni e Campionamento 

In [144]:
df = pd.DataFrame(np.arange(5*7).reshape((5,7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [ ]:
# permutazioni del dataframe originale 
sampler = np.random.permutation(5)
df.take(sampler)

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6


In [146]:
df.iloc[sampler]

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6


In [147]:
df.take(sampler, axis = 'columns')

,4,2,1,3,0
0,4,2,1,3,0
1,11,9,8,10,7
2,18,16,15,17,14
3,25,23,22,24,21
4,32,30,29,31,28


In [148]:
# campionamento (sampling)
df.sample(n=3)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20


In [149]:
# campionamento con rimpiazzo 
df.sample(n=3, replace = True)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27


- Calcol di dummy variable

In [156]:
df = pd.DataFrame({
    'key' : ['b', 'b', 'a', 'c', 'a', 'b'],
    'data1' : range(6)
})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [159]:
# one-hot encoding
dummies = pd.get_dummies(df['key'], prefix = 'key', dtype = np.int8)
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


## Handling 

In questa sezione parleremo di problemi di gestione dei dati generici, ma che pero trovano applicazione nella maggior parte dei progetti di data analisi. Parleremo di: 
- **Gestione dei Dati Mancanti**
- **Gestione delle Stringhe**
- **Gestione di Dati Categorici**

### Gestione dei dati mancanti 

La gestione dei dati mancanti prende il nome di **imputazione**. Molti metodi diversi possono essere impiegati per risolvere questo problema. In questo caso vedremo come:
- Eliminare le entrate di un asse dove ci sono dei valori *nulli*,
- Sostituire i valori nulli con un valore specifico.

In [2]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [4]:
string_data = pd.Series(['aaardvak', np.nan, None, 'avocado'])
string_data

0    aaardvak
1         NaN
2        None
3     avocado
dtype: object

In [9]:
data = pd.DataFrame(
    [[1., 6.5, 3.], [1., np.nan, np.nan],
    [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]]
)
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [3]:
# identificazione con maschera delle entrate nulle
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
# identificazione di diverse sentinelle per il valore nullo
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [6]:
# eliminare le entrate con valori nulli 
string_data.dropna()

0    aaardvak
3     avocado
dtype: object

In [10]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [11]:
data.dropna(how = 'all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [12]:
data.dropna(how = 'all', axis = 'columns')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
# filtraggio con maschera dei valori nulli 
string_data[string_data.notna()]

0    aaardvak
3     avocado
dtype: object

Se invece si volessero escludere solamente le entrate con un numero maggiore di una certa solglia di valori mancanti, basta specificarlo come argomento della funzione.  

In [13]:
np.random.seed(42)
df = pd.DataFrame(np.random.standard_normal((7,3)))
df

,0,1,2
0,0.496714,-0.138264,0.647689
1,1.523030,-0.234153,-0.234137
2,1.579213,0.767435,-0.469474
3,0.542560,-0.463418,-0.465730
4,0.241962,-1.913280,-1.724918
5,-0.562288,-1.012831,0.314247
6,-0.908024,-1.412304,1.465649


In [17]:
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,0.496714,NaN,NaN
1,1.523030,NaN,NaN
2,1.579213,NaN,-0.469474
3,0.542560,NaN,-0.465730
4,0.241962,-1.913280,-1.724918
5,-0.562288,-1.012831,0.314247
6,-0.908024,-1.412304,1.465649


In [18]:
df.dropna(thresh = 2)

,0,1,2
2,1.579213,NaN,-0.469474
3,0.542560,NaN,-0.465730
4,0.241962,-1.913280,-1.724918
5,-0.562288,-1.012831,0.314247
6,-0.908024,-1.412304,1.465649


Se invece si volessero sostituire i valori mancanti con un valore specifico e comunque possibile utilizzare l'API di Python. 

In [ ]:
# sostituzione di tutti i valori
df.fillna(0)

,0,1,2
0,0.496714,0.000000,0.000000
1,1.523030,0.000000,0.000000
2,1.579213,0.000000,-0.469474
3,0.542560,0.000000,-0.465730
4,0.241962,-1.913280,-1.724918
5,-0.562288,-1.012831,0.314247
6,-0.908024,-1.412304,1.465649


In [20]:
# sostituzione dei valori mancanti per colonna
df.fillna({1:0.5, 2:0})

,0,1,2
0,0.496714,0.500000,0.000000
1,1.523030,0.500000,0.000000
2,1.579213,0.500000,-0.469474
3,0.542560,0.500000,-0.465730
4,0.241962,-1.913280,-1.724918
5,-0.562288,-1.012831,0.314247
6,-0.908024,-1.412304,1.465649


In [21]:
np.random.seed(42)
df = pd.DataFrame(np.random.standard_normal((7,3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,0.496714,NaN,NaN
1,1.523030,NaN,NaN
2,1.579213,NaN,-0.469474
3,0.542560,NaN,-0.465730
4,0.241962,-1.913280,-1.724918
5,-0.562288,-1.012831,0.314247
6,-0.908024,-1.412304,1.465649


In [ ]:
# backward fill
df.bfill()

,0,1,2
0,0.496714,-1.913280,-0.469474
1,1.523030,-1.913280,-0.469474
2,1.579213,-1.913280,-0.469474
3,0.542560,-1.913280,-0.465730
4,0.241962,-1.913280,-1.724918
5,-0.562288,-1.012831,0.314247
6,-0.908024,-1.412304,1.465649


In [25]:
# forward fill
df.ffill()

,0,1,2
0,0.496714,NaN,NaN
1,1.523030,NaN,NaN
2,1.579213,NaN,-0.469474
3,0.542560,NaN,-0.465730
4,0.241962,-1.913280,-1.724918
5,-0.562288,-1.012831,0.314247
6,-0.908024,-1.412304,1.465649


In generale il metodo `fillna()` permette di gestire il problema dei valori mancanti in molti modi diversi. Si consglia di consultare la [documentazione](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) per ulteriori dettagli.

### Gestione delle Stringhe 

In Python esistono delle funzioni native che permettono di gestire le stringhe in maniera veloce. Una lista completa puo essere trovata al seguente [link](https://www.w3schools.com/python/python_ref_string.asp).

Un metodo molto utilizzato per gestire le stringhe e quello delle espressioni regolari (*regex*). Il module `re` di Python permette di lavorare con le espressioni regolari in maniera semplice. Queste possono essere utilizate per: 
1. pattern matching
2. sostituzione
3. splitting

In [ ]:
import re 

# splitting
text = 'foo bar\t baz \tqux'
re.split(r'\s+', text) # \s is the whitespace regex

['foo', 'bar', 'baz', 'qux']

In [27]:
# compiling the regex
regex = re.compile(r'\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [ ]:
# pattern matching: a list of all string matching the regex
regex.findall(text)

[' ', '\t ', ' \t']

In [ ]:
# strict matching: returns a matching only if the match is at the beginning of the string
regex.match(text)

In [30]:
# first matching: returns only the first string matching 
regex.search(text)

<re.Match object; span=(3, 4), match=' '>

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9._]+\.[A-Z]{2,4}" # mail regex
regex = re.compile(pattern, flags = re.IGNORECASE) # case-sensitive regex

regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [33]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [34]:
print(regex.match(text))

None


In [35]:
# substitution 
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [42]:
# segmentation 
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9._]+)\.([A-Z]{2,4})" # mail regex segmented
regex = re.compile(pattern, flags = re.IGNORECASE)

In [43]:
m = regex.match("wesm@bright.net")
m.groups()

('wesm', 'bright', 'net')

In [44]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [45]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



Parliamo adesso delle funzioni native di *pandas* per la gestione delle stringhe. 

In [46]:
data = {
    'Dave': 'dave@google.com',
    'Steve': 'steve@gmail.com',
    'Rob': 'rob@gmail.com',
    'Wes': np.nan
}

In [49]:
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [50]:
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [51]:
# matching 
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [52]:
data_as_string_text = data.astype('string')
data_as_string_text.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

In [54]:
# combinare pandas con re 
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9._]+)\.([A-Z]{2,4})" 
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [ ]:
# using the str attribute to do vectorized search
matches = data.str.findall(pattern, flags = re.IGNORECASE).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [56]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

In [ ]:
# requiring a DataFrame
data.str.extract(pattern, flags = re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


Per ulteriori informazioni riguardo all'utilizzo dell'operatore `str` delle `Series` si faccia riferimento alla [documentazione](https://pandas.pydata.org/docs/user_guide/text.html)

### Gestione di Dati Categorici 

In questa sezione parleremo del tipo `Categorical` di *pandas*. Nella libreria sono presente delle funzioni di utility per la gestione dei dati categorici che aiutano, ad esempio, per lo *one-hot encoding*. 

In [59]:
values = pd.Series(['apple', 'orange', 'apple', 'apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [60]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [61]:
pd.value_counts(values)

/tmp/ipykernel_7877/3297668723.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(values)


apple     6
orange    2
Name: count, dtype: int64

In [62]:
# data warehousing 
values = pd.Series([0, 1, 0, 0]*2)
dim = pd.Series(['apple', 'orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [ ]:
# encoding 
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [66]:
# Categorical type for handling encoding
fruits = dim.take(values)
fruits

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [82]:
N = len(fruits)
rng = np.random.default_rng(seed = 42)
df = pd.DataFrame({
    'fruit':fruits,
    'basket_ID':np.arange(N),
    'count':rng.integers(3,15, size = N),
    'weight': rng.uniform(0,4, size = N)},
    columns = ['basket_ID', 'fruit', 'count', 'weight']
)
df

,basket_ID,fruit,count,weight
0,0,apple,4,0.376709
1,1,orange,12,3.902489
0,2,apple,10,3.044559
0,3,apple,8,3.144257
0,4,apple,8,0.512455
1,5,orange,13,1.801544
0,6,apple,4,1.483192
0,7,apple,11,3.707060


In [ ]:
# category object
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [ ]:
# original array is stored as an attribute
c = fruit_cat.array
type(c)

pandas.core.arrays.categorical.Categorical

In [ ]:
# categories are stored as an Index attribitute
c.categories

Index(['apple', 'orange'], dtype='object')

In [76]:
# encoded array wrt to the category index is encoded as another attribute
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [ ]:
# mapping value-category 
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [ ]:
# casting a column to category type: obtaining all utilities
df['fruit'] = df['fruit'].astype('category')
df['fruit']

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [85]:
# creating a Categorical
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [86]:
# from_code constructor 
categories = ['foo', 'baz', 'bar']
codes = [0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'baz', 'bar', 'foo', 'foo', 'baz']
Categories (3, object): ['foo', 'baz', 'bar']

In [ ]:
# ordered constructor 
my_cats_3 = pd.Categorical.from_codes(codes, categories, ordered = True)
my_cats_3

['foo', 'baz', 'bar', 'foo', 'foo', 'baz']
Categories (3, object): ['foo' < 'baz' < 'bar']

# Esercizi 

## Data Transformation: Modifica e Rimozione
Questa sezione si concentra sulla ridenominazione degli assi, il rimpiazzo dei valori, e la gestione di duplicati e outlier.

🧠 Domande di Teoria

1. A cosa serve il metodo df.rename() e quale argomento devi usare per rinominare le colonne?

2. Quando è preferibile usare df.replace() rispetto a df.fillna()?

3. Qual è lo scopo del metodo df.duplicated() e come viene tipicamente utilizzato in combinazione con df.drop_duplicates()?

4. Spiega il concetto di Outlier in un set di dati. Qual è la tecnica del Clustering (Clamping) e quale funzione di NumPy viene utilizzata nel notebook per implementarla?

5. Qual è la differenza fondamentale tra l'utilizzo di pd.cut() e pd.qcut() per il binning dei dati?

👨‍💻 Esercizi di Programmazione

**Esercizio 1.1: Ridenominazione e Mappatura**

Dato il seguente DataFrame con etichette scritte in maiuscolo:

In [ ]:
data_es1 = pd.DataFrame(
    np.arange(6).reshape((3, 2)),
    index=['RIGA_A', 'RIGA_B', 'RIGA_C'],
    columns=['COL_1', 'COL_2']
)

1. Usa il metodo .rename() per rinominare l'indice 'RIGA_A' in 'Nuova_A' e la colonna 'COL_1' in 'Valore_Iniziale'.

2. In un'unica operazione, usa la funzione nativa Python str.lower (passata come argomento a rename o map) per convertire tutti i restanti nomi di indice e colonna in minuscolo.

**Esercizio 1.2: Rimpiazzo e Mapping dei Valori**

Dati i seguenti dati:

In [160]:
data_es2 = pd.DataFrame({
    'codice_errore': [101, 200, 999, 101, 404, 999],
    'stato': ['Attivo', 'Passivo', 'Sconosciuto', 'Attivo', 'Sconosciuto', 'Sconosciuto']
})

mappa_codici = {101: 'INFO', 200: 'OK', 999: 'ERRORE'}

1. Utilizza la mappa mappa_codici e il metodo .map() per creare una nuova colonna chiamata tipo_messaggio basata sulla colonna codice_errore.

2. Sostituisci tutti i valori 999 e 404 nella colonna codice_errore con np.nan in un'unica chiamata a .replace().

**Esercizio 1.3: Rimozione dei Duplicati**

Usando il DataFrame dell'esercizio 1.2 dopo le sostituzioni (con np.nan):

1. Identifica quali righe sono duplicate (True se duplicate, False altrimenti) considerando l'intero DataFrame.

2. Rimuovi i duplicati, mantenendo la prima occorrenza.

3. Aggiungi una nuova colonna fittizia voto = [1, 2, 3, 4, 5, 6]. Rimuovi ora i duplicati considerando solo le colonne codice_errore e stato, ma mantieni l'ultima occorrenza.

**Esercizio 1.4: Binning (qcut)**

Data una serie di voti di esame:

In [ ]:
voti = pd.Series([22, 30, 28, 18, 25, 29, 21, 24, 27, 26])

Usa pd.qcut() per dividere i voti in 3 quantili (terzili).

1. Assegna etichette testuali personalizzate ai terzili: ['Basso', 'Medio', 'Alto'].

2. Conta quanti elementi cadono in ciascuna categoria.

✅ Soluzioni

In [ ]:
import pandas as pd
import numpy as np

# Soluzione 1.1
data_es1 = pd.DataFrame(
    np.arange(6).reshape((3, 2)),
    index=['RIGA_A', 'RIGA_B', 'RIGA_C'],
    columns=['COL_1', 'COL_2']
)

# 1. Rename specifico
rinomina_mappa = {'RIGA_A': 'Nuova_A'}
rinomina_cols = {'COL_1': 'Valore_Iniziale'}
df_rinominato = data_es1.rename(index=rinomina_mappa, columns=rinomina_cols)
print("1. Ridenominazione specifica:")
print(df_rinominato)

# 2. Lowercase mapping (sugli assi rimanenti)
df_rinominato.columns = df_rinominato.columns.map(str.lower)
df_rinominato.index = df_rinominato.index.map(str.lower)
print("\n2. Conversione a minuscolo (metodo map):")
print(df_rinominato)
print("\n")

# Soluzione 1.2
data_es2 = pd.DataFrame({
    'codice_errore': [101, 200, 999, 101, 404, 999],
    'stato': ['Attivo', 'Passivo', 'Sconosciuto', 'Attivo', 'Sconosciuto', 'Sconosciuto']
})
mappa_codici = {101: 'INFO', 200: 'OK', 999: 'ERRORE'}

# 1. Mapping
data_es2['tipo_messaggio'] = data_es2['codice_errore'].map(mappa_codici)
print("1. Mapping con .map():")
print(data_es2)

# 2. Rimpiazzo con np.nan
data_es2['codice_errore'] = data_es2['codice_errore'].replace([999, 404], np.nan)
print("\n2. Rimpiazzo con np.nan:")
print(data_es2)
print("\n")

# Soluzione 1.3
# 1. Identificazione duplicati
print("1. Righe duplicate (sull'intero DF):")
print(data_es2.duplicated())

# 2. Rimozione duplicati (keep='first')
df_no_dup = data_es2.drop_duplicates(keep='first')
print("\n2. Rimozione duplicati (keep='first'):")
print(df_no_dup)

# 3. Rimozione duplicati per subset (keep='last')
data_es2['voto'] = [1, 2, 3, 4, 5, 6]
df_subset_last = data_es2.drop_duplicates(subset=['codice_errore', 'stato'], keep='last')
print("\n3. Rimozione duplicati su subset (keep='last'):")
print(df_subset_last)
print("\n")

# Soluzione 1.4
voti = pd.Series([22, 30, 28, 18, 25, 29, 21, 24, 27, 26])
nomi_terzili = ['Basso', 'Medio', 'Alto']

# 1. e 2. qcut con 3 quantili e labels
voti_terzili = pd.qcut(voti, q=3, labels=nomi_terzili)
print("1. e 2. Voti divisi in terzili:")
print(voti_terzili)

# 3. Conteggio
print("\n3. Conteggio per categoria:")
print(voti_terzili.value_counts())

## Handling: Dati Mancanti, Stringhe e Categorici

Questa sezione si concentra sull'imputazione dei valori mancanti, l'uso delle espressioni regolari e la gestione dei dati categorici.

🧠 Domande di Teoria

1. Qual è il concetto di Imputazione? Qual è la differenza tra usare df.dropna(how='any') e df.dropna(how='all')?

2. Spiega il funzionamento di df.bfill() (Backward Fill) e df.ffill() (Forward Fill) nel contesto dell'imputazione.

3. A cosa serve il modulo re di Python? Quale funzione è usata per trovare tutte le corrispondenze di un pattern in una stringa e quale per dividere una stringa in base a un pattern?

4. Nel contesto della gestione delle stringhe in Pandas, cosa aggiunge l'accessor .str alle Series che contengono stringhe?

5. Qual è il vantaggio di convertire una colonna di oggetti (stringhe) in tipo category in Pandas, specialmente per set di dati molto grandi?

👨‍💻 Esercizi di Programmazione

**Esercizio 2.1: Imputazione Dati Mancanti**

Riprendi il DataFrame df_nan dall'Esercizio 3.4 del modulo precedente, ma con indici espliciti per chiarezza:

In [ ]:
df_mancanti = pd.DataFrame({
    'A': [1.0, 2.0, np.nan, 4.0, 5.0],
    'B': [6.0, np.nan, np.nan, 9.0, 10.0],
    'C': [11.0, 12.0, 13.0, 14.0, np.nan]
}, index=['r1', 'r2', 'r3', 'r4', 'r5'])

Conta quanti valori non-NaN ci sono in ogni riga.

1. Utilizza il metodo .fillna() per sostituire i valori mancanti nella colonna 'B' con la media (.mean()) della colonna 'B'.

2. Crea un nuovo DataFrame eliminando tutte le colonne che contengono più di due valori mancanti (usa dropna() con il parametro thresh).

**Esercizio 2.2: Gestione Avanzata delle Stringhe con Regex**

Dati i seguenti indirizzi email:

In [ ]:
emails = pd.Series([
    'alice.rossi@universita.edu',
    'bruno_verdi@azienda.com',
    'chiara-bianchi@posta.it',
    'NaN'
])
pattern = r'([\w\.-]+)@(\w+)\.(it|com|edu)' # Pattern per Username, Dominio, Suffisso

1. Utilizza l'accessor .str con .extract() per estrarre lo Username, il Dominio e il Suffisso in colonne separate.

2. Utilizza .str.contains() per creare una maschera booleana che indichi quali email appartengono al dominio 'universita.edu' o 'posta.it'.

**Esercizio 2.3: Conversione a Categorical**

Dato il seguente set di dati con categorie non ordinate:

In [ ]:
livelli = pd.Series(['Basso', 'Medio', 'Alto', 'Medio', 'Alto', 'Basso', 'Basso'])

Converti la Series livelli nel tipo category.

1. Accedi all'attributo .codes della Series categorica. Cosa rappresentano i numeri nell'array codes?

2. Modifica la Series in modo che l'ordine delle categorie sia ordinato (ordered) come ['Basso', 'Medio', 'Alto'].

✅ Soluzioni

In [ ]:
import pandas as pd
import numpy as np
import re

# Soluzione 2.1
df_mancanti = pd.DataFrame({
    'A': [1.0, 2.0, np.nan, 4.0, 5.0],
    'B': [6.0, np.nan, np.nan, 9.0, 10.0],
    'C': [11.0, 12.0, 13.0, 14.0, np.nan]
}, index=['r1', 'r2', 'r3', 'r4', 'r5'])

print("--- Soluzione 2.1 ---")
print("DataFrame originale con NaN:")
print(df_mancanti)

# 1. Conteggio non-NaN per riga
print("\n1. Conteggio Non-NaN per riga:")
print(df_mancanti.count(axis='columns'))

# 2. Fillna con la media della colonna B
media_B = df_mancanti['B'].mean()
df_imputato_B = df_mancanti.fillna({'B': media_B})
print(f"\n2. Imputazione colonna B con media ({media_B:.2f}):")
print(df_imputato_B)

# 3. Dropna con soglia (thresh) sulle colonne
# Ci sono 5 righe, quindi per eliminare una colonna deve avere <= 2 valori non nulli.
# Se si intende eliminare le righe con più di 2 valori mancanti, il codice è df_mancanti.dropna(thresh=3)
# Poiché l'esercizio è sui dati mancanti in generale, implementiamo la rimozione delle righe
df_pulito = df_mancanti.dropna(thresh=3) # Mantieni le righe con almeno 3 valori non nulli
print("\n3. Righe mantenute con almeno 3 valori non-NaN (thresh=3):")
print(df_pulito)
print("\n")

# Soluzione 2.2
emails = pd.Series([
    'alice.rossi@universita.edu',
    'bruno_verdi@azienda.com',
    'chiara-bianchi@posta.it',
    'NaN'
])
pattern = r'([\w\.-]+)@(\w+)\.(it|com|edu)' 

print("--- Soluzione 2.2 ---")
# 1. Estrazione in colonne separate
email_estratti = emails.str.extract(pattern, flags=re.IGNORECASE)
email_estratti.columns = ['Username', 'Dominio', 'Suffisso']
print("1. Estrazione email con Regex:")
print(email_estratti)

# 2. Maschera booleana
maschera_domini = emails.str.contains('universita.edu|posta.it')
print("\n2. Maschera per 'universita.edu' o 'posta.it':")
print(maschera_domini)
print("\n")

# Soluzione 2.3
livelli = pd.Series(['Basso', 'Medio', 'Alto', 'Medio', 'Alto', 'Basso', 'Basso'])

print("--- Soluzione 2.3 ---")
# 1. Conversione in tipo category
livelli_cat = livelli.astype('category')
print("1. Series convertita in 'category':")
print(livelli_cat)

# 2. Accesso a .codes
print("\n2. Attributo .codes:")
print(livelli_cat.cat.codes)

# 3. Modifica per ordinamento
nuovo_ordine = ['Basso', 'Medio', 'Alto']
livelli_cat_ordinato = livelli_cat.cat.set_categories(nuovo_ordine, ordered=True)
print("\n3. Series Categorical ordinata:")
print(livelli_cat_ordinato)